***
<h2>Example of face detection with a vggface2 model</h2>

***



In [1]:
import keras_applications
from keras import backend as K
#from keras_applications import vgg16
import keras_vggface
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
import os
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [10]:
# Example of face detection with a vggface2 model

 
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# extract faces and calculate face embeddings for a list of photo files
filenames = os.listdir('known/')
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat
 
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
        

# running the inference:
path = 'known/'
filenames = []
for filename in os.listdir('known/'):
    filenames.append(os.path.join(path,filename))
# get embeddings file filenames
embeddings = get_embeddings(filenames)

for i in range (0,embeddings.shape[0]):
    print(os.listdir('known/')[0], 'Vs',os.listdir('known/')[i])
    is_match(embeddings[0], embeddings[i])
    

messi_1.jpg Vs messi_1.jpg
>face is a Match (0.000 <= 0.500)
messi_1.jpg Vs messi_2.jpg
>face is a Match (0.293 <= 0.500)
messi_1.jpg Vs messi_3.jpg
>face is a Match (0.322 <= 0.500)
messi_1.jpg Vs messi_6.jpg
>face is a Match (0.417 <= 0.500)
messi_1.jpg Vs messi_7.jpg
>face is a Match (0.272 <= 0.500)
messi_1.jpg Vs ronaldo_1.jpg
>face is NOT a Match (0.690 > 0.500)
messi_1.jpg Vs ronaldo_2.jpg
>face is NOT a Match (0.695 > 0.500)
messi_1.jpg Vs ronaldo_3.jpg
>face is NOT a Match (0.726 > 0.500)
messi_1.jpg Vs ronaldo_5.jpg
>face is NOT a Match (0.695 > 0.500)
messi_1.jpg Vs ronaldo_6.jpg
>face is NOT a Match (0.639 > 0.500)
messi_1.jpg Vs ronaldo_7.jpg
>face is NOT a Match (0.706 > 0.500)
